# May need multithread to download pictrue and train

## Multithread using `thread.start_new_thread(...)`

In [1]:
import thread
import time

def print_time(threadName, delay):
    count = 0
    while count < 5:
        time.sleep(delay)
        count += 1
        print '{}: {}'.format(threadName, time.ctime(time.time()))
        
try:
    thread.start_new_thread(print_time, ('Thread_1', 2, ))
    thread.start_new_thread(print_time, ('Thread_2', 4, ))
except:
    print 'Error: unable to start thread'
    
while 1:
    pass

Thread_1: Mon Feb  6 04:06:24 2017
Thread_1: Mon Feb  6 04:06:26 2017Thread_2: Mon Feb  6 04:06:26 2017

Thread_1: Mon Feb  6 04:06:28 2017
Thread_2: Mon Feb  6 04:06:30 2017
Thread_1: Mon Feb  6 04:06:30 2017
Thread_1: Mon Feb  6 04:06:32 2017
Thread_2: Mon Feb  6 04:06:34 2017


KeyboardInterrupt: 

## Multithread by creating subclass of `threading.Thread`

In [4]:
import threading
import time

exitFlag = 0

class myThread(threading.Thread):
    def __init__(self, threadID, name, counter):
        threading.Thread.__init__(self)
        self.name = name
        self.counter = counter
    def run(self):
        print 'starting ' + self.name
        print_time(self.name, self.counter, 5)
        print 'existing ' + self.name
        
def print_time(threadName, delay, counter):
    while counter:
        if exitFlag:
            threadName.exit()
        time.sleep(delay)
        print '{}: {}'.format(threadName, time.ctime(time.time()))
        counter -= 1
            
# Creat new threads
thread1 = myThread(1, 'Thread-1', 1)
thread2 = myThread(2, 'Thread-2', 2)

# Start new threads
thread1.start()
thread2.start()

print 'Exiting Main Thread'

starting Thread-1
Exiting Main Threadstarting Thread-2

Thread-1: Mon Feb  6 04:23:27 2017
Thread-2: Mon Feb  6 04:23:28 2017Thread-1: Mon Feb  6 04:23:28 2017

Thread-1: Mon Feb  6 04:23:29 2017
Thread-2: Mon Feb  6 04:23:30 2017
Thread-1: Mon Feb  6 04:23:30 2017
Thread-1: Mon Feb  6 04:23:31 2017
existing Thread-1
Thread-2: Mon Feb  6 04:23:32 2017
Thread-2: Mon Feb  6 04:23:34 2017
Thread-2: Mon Feb  6 04:23:36 2017
existing Thread-2


## Synchronizing

In [16]:
import threading
import time

class myThread(threading.Thread):
    def __init__(self, threadID, name, counter):
        threading.Thread.__init__(self)
        self.threadID = threadID
        self.name = name
        self.counter = counter
    def run(self):
        print 'starting '+ self.name
        # get the lock to synchronize threads
        threadLock.acquire()
        print_time(self.name, self.counter, 3)
        # free lock to release next thread
        threadLock.release()
        
def print_time(threadName, delay, counter):
    while counter:
        time.sleep(delay)
        print '{}: {}'.format(threadName, time.ctime(time.time()))
        counter -= 1
        
threadLock = threading.Lock()
threads = []

# creating new threads
thread1 = myThread(1, "Thread-1", 1)
thread2 = myThread(2, 'Thread-2', 2)

# start new threads
thread1.start()
thread2.start()

# add threads to threads list
threads.append(thread1)
threads.append(thread2)

# wait for all threads to complete
for t in threads:
    t.join()
print 'Exiting Main Thread'

starting Thread-1
starting Thread-2
Thread-1: Mon Feb  6 04:57:22 2017
Thread-1: Mon Feb  6 04:57:23 2017
Thread-1: Mon Feb  6 04:57:24 2017
Thread-2: Mon Feb  6 04:57:26 2017
Thread-2: Mon Feb  6 04:57:28 2017
Thread-2: Mon Feb  6 04:57:30 2017
Exiting Main Thread


In [22]:
import Queue
import threading
import time

exitFlag = 0

class myThread(threading.Thread):
    def __init__(self, threadID, name, q):
        threading.Thread.__init__(self)
        self.threadID = threadID
        self.name = name
        self.q = q
    def run(self):
        print 'starting ' + self.name
        processing_data(self.name, self.q)
        print 'exiting ' + self.name

def processing_data(threadName, q):
    while not exitFlag:
        queueLock.acquire()
        if not workQueue.empty():
            data = q.get()
            queueLock.release()
            print '{} processing {}'.format(threadName, data)
        else:
            queueLock.release()
        time.sleep(1)

threadList = ['Thread-1', 'Thread-2', 'Thread-3']
nameList = ['One', 'Two', 'Three', 'Four', 'Five']
queueLock = threading.Lock()
workQueue = Queue.Queue(10)
threads = []
threadID = 1

# create new threads
for tName in threadList:
    thread = myThread(threadID, tName, workQueue)
    thread.start()
    threads.append(thread)
    threadID += 1
    
# fill the queue
queueLock.acquire()
for word in nameList:
    workQueue.put(word)
queueLock.release()

# wait for the queue to empty
while not workQueue.empty():
    pass

# notify threads it is time to exit
exitFlag = 1

# wait for all threads to complete
for t in threads:
    t.join()
print 'Exiting Main Thread'

starting Thread-2starting Thread-1 starting Thread-3


Thread-1 processing One
 Thread-3 processing Two
Thread-3 processing Three
Thread-2 processing Four
Thread-1 processing Five
exiting Thread-1exiting Thread-3
exiting Thread-2

Exiting Main Thread
